# Scraping real state attributes in Benito Juarez, CDMX

In [1]:
from bs4 import BeautifulSoup
import requests
from scrapy import Selector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

#my functions
from rdb_functions import clean_list
from rdb_functions import clean_list_url


We manually searched real state sales in metroscubicos.com in Alcaldia Benito Juarez.

In [2]:
url='https://inmuebles.metroscubicos.com/casas/venta/distrito-federal/benito-juarez/#origin=search&as_word=true'
#Get html content
html = requests.get( url ).content
sel = Selector( text = html )

## Getting all urls

In [3]:
path='//*[@class="andes-pagination__link"]'
urls=sel.xpath(path).extract()

#Getting the fist urls
urls=clean_list_url(urls)

there aren't further urls beyond the last from the previous query

In [4]:
#starting in last position from the previous query
url=urls[len(urls)-1]

html = requests.get( url ).content
sel = Selector( text = html )

#Getting the lasts urls
path='//*[@class="andes-pagination__link"]'
urls2=sel.xpath(path).extract()
urls2=clean_list_url(urls2)
urls.extend(urls2)

## Getting Attributes

In [5]:
#empty list of attributes
atr=[]
price=[]
status=[]
adress=[]

for url in urls:
    try:
        html = requests.get( url ).content
        sel = Selector( text = html )
    except:
        print(f'This url didnt work: \n {url}')
              
    try:
        path='//*[@class="item__attrs"]'
        aux=sel.xpath(path).extract()
        atr.extend(clean_list(aux))
    except:
        print(f'no attributes info in:\n{url}')
        
    try:
        path='//*[@class="price__fraction"]'
        aux=sel.xpath(path).extract()
        price.extend(clean_list(aux))
    except:
        print(f'no prices info in:\n{url}')

    try:
        path='//*[@class="item__info-title"]'
        aux=sel.xpath(path).extract()
        status.extend(clean_list(aux))
    except:
        print(f'no prices info in:\n{url}')
        
    try:
        path='//*[@class="item__title"]'
        aux=sel.xpath(path).extract()
        adress.extend(clean_list(aux))
    except:
        print(f'no prices info in:\n{url}')  
        

This url didnt work: 
 #
This url didnt work: 
 #


## Generating the Pandas

In [6]:
data=pd.DataFrame({'attributes':atr,'price':price,'status':status,'adress':adress})

In [7]:
data.tail(10)

,attributes,price,status,adress
998,140 m² construidos | 2 recs.,"5,900,000",Casa en venta,Alhambra 1011 - Portales Sur - Benito Juárez ...
999,124 m² construidos | 3 recs.,"2,700,000",Casa en venta,Castilla - Benito Juárez - Distrito Federal
1000,247 m² construidos | 3 recs.,"15,219,666",Casa en venta,Amores - Del Valle Norte - Benito Juárez - Di...
1001,180 m² construidos | 3 recs.,"1,155,966",Casa en venta,CASTILLA 142 - Alamos - Benito Juárez - Distr...
1002,150 m² construidos | 3 recs.,"1,400,000",Casa en venta,MANUEL LOPEZ COTILLA 934 - Del Valle Centro -...
1003,180 m² construidos | 4 recs.,"1,352,036",Casa en venta,GABRIEL MANCER 46 - Del Valle Centro - Benito...
1004,150 m² construidos | 3 recs.,"5,990,000",Casa en venta,Jaca 914 - Santa Cruz Atoyac - Benito Juárez ...
1005,560 m² construidos | 4 recs.,"1,010,000",Casa en venta,IDAHO 00000 - Nápoles - Benito Juárez - Distr...
1006,200 m² construidos | 4 recs.,"1,049,998",Casa en venta,MARIA HERNANDEZ ZARCO 68 - Alamos - Benito Ju...
1007,219 m² construidos | 3 recs.,"13,686,875",Casa en venta,Dallas - Nápoles - Benito Juárez - Distrito F...


## Printing the results

In [8]:
data.to_csv('benito_juarez.csv',index=False)

In [ ]:
d2 = data['adress'] # Replacing values for later use
data['adress'] = [i.replace("-",",") for i in d2]

In [ ]:
splt_att = data["attributes"].str.split("|", n = 2, expand = True) # splitting atribute to get new columns
splt_att.columns = ['x','rooms']
s_a2 = splt_att["x"].str.split(" ", n = 3, expand = True)
s_a2.columns = ['index', 'm2', 'metric','state']

m2 = s_a2['m2']
state = s_a2['state']
rooms = splt_att['rooms']

clean_data = pd.DataFrame({'m2': m2, 
                           'State': state, 
                           'Rooms': rooms, 
                           'Price': data['price'], 
                           'Status': data['status'],
                           'Adress': data['adress']})

clean_data.head()

In [ ]:
clean_data.to_csv("./clean_data.csv") # printing to get coordinates

In [ ]:
# Data files, all of these census data was acquired from inegi.org.mx
Censo2010_ConjuntoDatos_path = "DATA_CENSOs/resultados_ageb_urbana_09_cpv2010.csv"
Censo2010_DiccionarioDatos_path = "DATA_CENSOs/fd_resultados_ageb_urbana_cpv2010.csv"
Censo2010_MetaDatos_path = "DATA_CENSOs/resultados_ageb_urbana_09_cpv2010"
Censo2020_ConjuntoDatos_path = "DATA_CENSOs/conjunto_de_datos_ageb_urbana_09_cpv2020.csv"
Censo2020_DiccionarioDatos_path = "DATA_CENSOs/diccionario_datos_ageb_urbana_09_cpv2020.csv"
Censo2020_MetaDatos_path = "DATA_CENSOs/metadatos_ageb_urbana_09_cpv2020"

# Read the data and the study results
Conjunto2010_metadata = pd.read_csv(Censo2010_ConjuntoDatos_path, encoding='ISO-8859-1')
Diccionario2010_metadata = pd.read_csv(Censo2010_DiccionarioDatos_path, encoding='ISO-8859-1')
Conjunto2020_metadata = pd.read_csv(Censo2020_ConjuntoDatos_path, encoding='ISO-8859-1')
Diccionario2020_metadata = pd.read_csv(Censo2020_DiccionarioDatos_path, encoding='latin1')


# Filter out AGEB in Benito Juarez

BenitoJuarez2010_metadata=Conjunto2010_metadata[Conjunto2010_metadata['nom_mun']=="Benito JuÃ¡rez"]
BenitoJuarez2020_metadata=Conjunto2020_metadata[Conjunto2020_metadata['NOM_MUN']=="Benito JuÃ¡rez"]


#format dfs for comparison
BenitoJuarez2010_metadata.columns=map(lambda x:str(x).upper(),BenitoJuarez2010_metadata.columns)



In [ ]:
#clean BJ2010 data, remove non repeating colums

BJ210=BenitoJuarez2010_metadata.drop(columns=['Ï»¿"ENTIDAD"','PCON_LIM','PCLIM_MOT','PCLIM_VIS','PCLIM_LENG','PCLIM_AUD','PCLIM_MOT2',
                                       'PCLIM_MEN','PCLIM_MEN2','PSIN_LIM','PDER_SEGP','PCATOLICA','PNCATOLICA',
                                       'POTRAS_REL','PSIN_RELIG'])

#sort for easier cleaning
cols=BJ210.columns.tolist()
cols.sort()
df1=BJ210[cols]
df1.insert(0,"Tiempo",(2010),True)
df1

In [ ]:
#clean BJ2020 data, remove non repeating colums

BJ220=BenitoJuarez2020_metadata.drop(columns=['ï»¿ENTIDAD','PCON_DISC','PCDISC_MOT','PCDISC_VIS','PCDISC_LENG','PCDISC_AUD','PCDISC_MOT2',
                                       'PCDISC_MEN','PCON_LIMI','PCLIM_CSB','PCLIM_VIS','PCLIM_HACO','PCLIM_OAUD',
                                       'PCLIM_MOT2','PCLIM_RE_CO','PCLIM_PMEN','PSIND_LIM','PAFIL_PDOM','PDER_SEGP',
                                       'PDER_IMSSB','PAFIL_IPRIV','PAFIL_OTRAI','PCATOLICA','PRO_CRIEVA','POTRAS_REL',
                                       'PSIN_RELIG','VPH_AEASP','VPH_TINACO','VPH_CISTER','VPH_LETR','VPH_NDEAED',
                                       'VPH_DSADMA','VPH_NDACMM','VPH_HMICRO','VPH_MOTO','VPH_BICI','VPH_STVP',
                                       'VPH_SPMVPI','VPH_CVJ','VPH_SINRTV','VPH_SINLTC','VPH_SINCINT','VPH_SINTIC',
                                        'POB_AFRO','POB_AFRO_F','POB_AFRO_M'])

BJ220

In [ ]:
#sort 2020 data
cols2=BJ220.columns.tolist()
cols2.sort()
df2=BJ220[cols2]
df2.insert(0,"Tiempo",(2020),True)
df2

In [ ]:
#what happens if we merge
merge_dfs=pd.merge(df1, df2,how='outer')
merge_dfs

In [ ]:
Final_df=merge_dfs.loc[merge_dfs.duplicated(subset=['AGEB','MZA'],keep=False),:]
Final_df

In [ ]:
BJClean_data=Final_df[Final_df['AGEB']!="0000"]


BJClean_data

In [ ]:
BJClean_data.to_csv('BJClean_data.csv')

In [ ]:
data2 = BJClean_data # change "data" name

In [ ]:
chosen_df = data2[['Tiempo','NOM_ENT', 'MUN', 'LOC', 'AGEB', 'MZA', 'POBTOT', 'P_0A2', 'P_60YMAS', 'PROM_HNV', 'GRAPROES', 
                  'GRAPROES_F', 'POCUPADA', 'POCUPADA_F', 'PDER_SS', 'VIVTOT', 'TVIVHAB', 'TVIVPAR', 'VIVPAR_HAB', 
                  'VIVPAR_UT', 'PROM_OCUP', 'VPH_2YMASD', 'VPH_AUTOM', 'VPH_PC', 'VPH_INTER']]
chosen_df.head()

In [ ]:
steps = chosen_df.replace(to_replace='*', value='') # replacing '*' with ''
steps = steps.replace(to_replace='N/D', value='') # replacing 'N/D' with ''
steps.columns= steps.columns.str.lower() # change column names to lower case

steps.head()

In [ ]:
steps.dtypes

In [ ]:
cols=[i for i in steps.columns if i not in ["nom_ent","ageb"]] # change data frame dtypes to do math
for col in cols:
    steps[col]=pd.to_numeric(steps[col])

steps.dtypes

In [ ]:
pd.options.display.max_columns = None # to show all columns
pd.options.display.max_rows = None

steps.head()

In [ ]:
list(steps) # to know column names

In [ ]:
# Percents 

pop02_pct = steps['p_0a2']/steps['pobtot']*100

steps['pct_0a2'] = pop02_pct

pop60_pct = steps['p_60ymas']/steps['pobtot']*100
steps['pct_60'] = pop60_pct

working_fem_pct = steps['pocupada_f']/steps['pocupada']*100

steps['pct_wf'] = working_fem_pct

inhabited_hh_pct = steps['tvivhab']/steps['vivtot']*100
steps['pct_t_inhab_hh'] = inhabited_hh_pct

private_hh_pct = steps['tvivpar']/steps['vivtot']*100
steps['pct_privhh'] = private_hh_pct

inhabited_pvt_hh_pct = steps['vivpar_hab']/steps['vivtot']*100
steps['pct_privhh_inhab'] = inhabited_pvt_hh_pct

temp_pvt_hh_pct = steps['vivpar_ut']/steps['vivtot']*100
steps['pct_hhtemp'] = temp_pvt_hh_pct

hh_with_car = steps['vph_autom']/steps['vivtot']*100
steps['pct_hh_car'] = hh_with_car

hh_with_pc = steps['vph_pc']/steps['vivtot']*100
steps['pct_hh_pc'] = hh_with_pc

hh_with_int = steps['vph_inter']/steps['vivtot']*100
steps['pct_hh_int'] = hh_with_int

shiny_df = steps
shiny_df.head()

In [ ]:
shiny_df.to_csv("./BJ_10_20.csv")